# Fase 1: EDA Técnico - F1 Dataset

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('f1_staging.db')
conn

In [3]:
tables = pd.read_sql_query("select name from sqlite_master WHERE type = 'table' order by name ", conn)
list_name = tables['name'].tolist()
list_name

['circuits',
 'constructor_results',
 'constructor_standings',
 'constructors',
 'driver_standings',
 'drivers',
 'lap_times',
 'pit_stops',
 'qualifying',
 'races',
 'results',
 'seasons',
 'sprint_results',
 'status']

In [4]:
def get_table(table_name):
    return pd.read_sql(f"SELECT * FROM {table_name}", conn)

In [5]:
results = get_table('results')
results.tail(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
26754,26760,1144,825,210,20,14,16,16,16,0.0,57,\N,\N,57,1,1:25.637,222.002,11
26755,26761,1144,859,215,30,12,17,17,17,0.0,55,\N,\N,52,12,1:28.751,214.212,5
26756,26762,1144,822,15,77,9,\N,R,18,0.0,30,\N,\N,14,19,1:29.482,212.462,130
26757,26763,1144,861,3,43,20,\N,R,19,0.0,26,\N,\N,5,17,1:29.411,212.631,5
26758,26764,1144,815,9,11,10,\N,R,20,0.0,0,\N,\N,\N,0,\N,\N,4


In [6]:
null_report = []
for table in list_name:
    df = get_table(table)
    n_counts = (df == '\\N').sum()
    n_counts = n_counts[n_counts > 0]
    for col, val in n_counts.items():
        null_report.append({'Tabla':table,'Columna':col,'Placeholder_Nulos': val})
df_nulls = pd.DataFrame(null_report)
print("Reporte de Pseudo-Nulos detectados:")
print(df_nulls)

Reporte de Pseudo-Nulos detectados:
                  Tabla          Columna  Placeholder_Nulos
0   constructor_results           status              12608
1               drivers           number                802
2               drivers             code                757
3            qualifying               q1                156
4            qualifying               q2               4625
5            qualifying               q3               6819
6                 races             time                731
7                 races         fp1_date               1035
8                 races         fp1_time               1057
9                 races         fp2_date               1035
10                races         fp2_time               1057
11                races         fp3_date               1053
12                races         fp3_time               1072
13                races       quali_date               1035
14                races       quali_time               1057
15  

In [7]:
nan_report = []
for table in list_name:
    df = get_table(table)
    nan_count = df.isna().sum()
    nan_count = nan_count[nan_count > 0]
    for col, val in nan_count.items():
        nan_report.append({'Table': table, 'Columna': col, 'Placeholder': val})
df_nan = pd.DataFrame(nan_report)
print("Reporte de Pseudo-Nulos detectados:")
print(df_nan)

Reporte de Pseudo-Nulos detectados:
        Table Columna  Placeholder
0  qualifying      q2           22
1  qualifying      q3           46


In [8]:
typ_report = []

for table in list_name:
    df = get_table(table)
    typ_count = df.dtypes
    for col, val in typ_count.items():
        typ_report.append({'Table': table, 'Columna': col, 'Tipo_dato': str(val)})
df_typ = pd.DataFrame(typ_report)
print("Reporte tipo de datos detectados:")
print(df_typ)

Reporte tipo de datos detectados:
              Table         Columna Tipo_dato
0          circuits       circuitId     int64
1          circuits      circuitRef    object
2          circuits            name    object
3          circuits        location    object
4          circuits         country    object
..              ...             ...       ...
115  sprint_results      fastestLap    object
116  sprint_results  fastestLapTime    object
117  sprint_results        statusId     int64
118          status        statusId     int64
119          status          status    object

[120 rows x 3 columns]


In [9]:
df_sospechosas = df_typ[df_typ['Tipo_dato'] == 'object']
print(df_sospechosas)

                     Table          Columna Tipo_dato
1                 circuits       circuitRef    object
2                 circuits             name    object
3                 circuits         location    object
4                 circuits          country    object
8                 circuits              url    object
13     constructor_results           status    object
19   constructor_standings     positionText    object
22            constructors   constructorRef    object
23            constructors             name    object
24            constructors      nationality    object
25            constructors              url    object
31        driver_standings     positionText    object
34                 drivers        driverRef    object
35                 drivers           number    object
36                 drivers             code    object
37                 drivers         forename    object
38                 drivers          surname    object
39                 drivers  

In [10]:
dup_report = []
for table in list_name:
    df = get_table(table)
    n_duplicados = df.duplicated().sum()
    if n_duplicados > 0:
        dup_report.append({'Tabla': table, 'Filas_Duplicadas': n_duplicados})

df_dups = pd.DataFrame(dup_report)
dup_report

[]

In [11]:
## RESUMEN
health_report = []

for table in list_name:
    df = get_table(table)

    n_nans = df.isna().sum().sum()
    n_pseudos = (df == '\\N').sum().sum()
    n_dups = df.duplicated().sum()
    cols_obj = (df.dtypes == 'object').sum()
    
    health_report.append({
        'Tabla': table,
        'Filas': len(df),
        'Columnas': len(df.columns),
        'Nulos_Reales': n_nans,
        'Pseudo_Nulos': n_pseudos,
        'Duplicados': n_dups,
        'Cols_Texto': cols_obj
    })

df_health = pd.DataFrame(health_report)
print("Matriz de Salud de los Datos:")
display(df_health)

Matriz de Salud de los Datos:


,Tabla,Filas,Columnas,Nulos_Reales,Pseudo_Nulos,Duplicados,Cols_Texto
0,circuits,77,9,0,0,0,5
1,constructor_results,12625,5,0,12608,0,1
2,constructor_standings,13391,7,0,0,0,1
3,constructors,212,5,0,0,0,4
4,driver_standings,34863,7,0,0,0,1
5,drivers,861,9,0,1559,0,8
6,lap_times,589081,6,0,0,0,1
7,pit_stops,11371,7,0,0,0,2
8,qualifying,10494,9,68,11600,0,3
9,races,1125,18,0,11349,0,14
